In [1]:
from pyspark.sql import SparkSession, functions as F
from delta import configure_spark_with_delta_pip
from pyspark.sql.functions import col, trim, lower, regexp_replace
import os

In [2]:
mongo_connector_jar = "/home/provira/Documents/TFM/TFM/src/P2/Explotation Zone/jars/mongo-spark-connector_2.12-3.0.1.jar"
mongo_driver_jar = "/home/provira/Documents/TFM/TFM/src/P2/Explotation Zone/jars/mongo-java-driver-3.12.10.jar"

In [ ]:
builder = SparkSession.builder \
    .appName("Trusted_Zone") \
    .config("spark.jars", f"{mongo_connector_jar},{mongo_driver_jar}") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.mongodb.read.connection.uri", "mongodb://localhost:27017") \
    .config("spark.mongodb.write.connection.uri", "mongodb://localhost:27017")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

25/06/27 16:56:21 WARN Utils: Your hostname, provira-ERAZER-P6705-MD61203 resolves to a loopback address: 127.0.1.1; using 192.168.1.55 instead (on interface wlo1)
25/06/27 16:56:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/provira/anaconda3/envs/spark_py3.9/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/provira/.ivy2/cache
The jars for the packages stored in: /home/provira/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d9c7aa2b-7a07-4d24-9df5-3144cf72c639;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 738ms :: artifacts dl 28ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-r

In [9]:
df = spark.read \
    .format("mongo") \
    .option("uri", "mongodb://localhost:27017/tfm.tf-idf") \
    .load()


# JOIN - Positive Emotions

In [10]:
good_emotions = [
    "happy",
    "joy",
    "love",
    "excited",
    "relieved",
    "grateful",
    "content",
    "proud",
    "hopeful",
    "amused"
]

bad_emotions = [
    "sad",
    "angry",
    "fear",
    "disgust",
    "anxious",
    "jealous",
    "frustrated",
    "guilty",
    "ashamed",
    "lonely"
]

In [13]:
print(df.columns)
df_positive = df.filter(df.Emotion.isin(good_emotions))
df_positive.show(5)


['Emotion', '_id', 'filtered_words', 'raw_features_array', 'stemmed_words', 'text', 'tfidf_features_array', 'words']
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Emotion|                 _id|      filtered_words|  raw_features_array|       stemmed_words|                text|tfidf_features_array|               words|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   love|{685ea7590c4b1155...|[feel, jealous, b...|[1.0, 0.0, 0.0, 0...|[feel, jealous, b...|i feel jealous be...|[0.02426812797784...|[i, feel, jealous...|
|   love|{685ea7590c4b1155...|[feel, like, take...|[1.0, 1.0, 0.0, 0...|[feel, like, take...|i feel like she h...|[0.02426812797784...|[i, feel, like, s...|
|   love|{685ea7590c4b1155...|[feel, like, im, ...|[1.0, 1.0, 1.0, 0...|[feel, like, im, ...|i fee

In [16]:
df_positive.select(
    "text", "Emotion", "words", "filtered_words", "stemmed_words",
    "raw_features_array", "tfidf_features_array"
).write \
    .format("mongo") \
    .option("uri", "mongodb://localhost:27017/tfm_trsuted_zone.join_positive_emotions") \
    .mode("append") \
    .save()